<a href="https://colab.research.google.com/github/kimwoonggon/publicservant_AI/blob/master/1_(HuggingFace%2BTF2)%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%ED%8F%89%EA%B0%80_%EA%B8%8D%EB%B6%80%EC%A0%95_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# python 3.11.8
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [19]:
X, y = [], []
max_length = 0
f = open("D:\\programming\\cloud-term-project\\review_with_label2.txt", 'r', encoding='utf-8')
for line in f.readlines():
    line = line.rstrip().split(',')
    if line[0] == '':
        continue
    X.append(','.join(line[:-1]))
    if len(X[-1]) > max_length:
        max_length = len(X[-1])

    y.append(int(line[-1]))
print(max_length)

from sklearn.model_selection import train_test_split
import time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=int(time.time())%1000)
print(len(X_train))


train = pd.DataFrame({'document':X_train, 'label':y_train})
test = pd.DataFrame({'document':X_test, 'label':y_test})

205
22038


In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

d:\programming\cloud-term-venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\tokenizer.json
loading configuration file config.json from cache a

In [29]:
def convert_data(data_df):
    global tokenizer

    SEQ_LEN = 32 #SEQ_LEN : 버트에 들어갈 인풋의 길이

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')

        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

BATCH_SIZE = 20
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "document"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

100%|██████████| 22038/22038 [00:05<00:00, 3849.44it/s]


In [30]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

100%|██████████| 5510/5510 [00:01<00:00, 3593.33it/s]


In [31]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
SEQ_LEN = 32
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])
# 버트 아웃풋의 텐서의 shape은 [batch_size, 문장의 길이, 768]임

d:\programming\cloud-term-venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_h

In [32]:
bert_outputs = bert_outputs[1]
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1.0e-5), loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [33]:
sentiment_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 32)]         0           []                               
                                                                                                  
 input_segment (InputLayer)     [(None, 32)]         0           []                               
                                                                                                  
 tf_bert_model_4 (TFBertModel)  TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]',      

In [34]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
opt = tfa.optimizers.RectifiedAdam(lr=1.0e-5, weight_decay=0.0025, warmup_proportion=0.05)

d:\programming\cloud-term-venv\Lib\site-packages\tensorflow_addons\optimizers\rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [35]:
def create_sentiment_bert():
  # 버트 pretrained 모델 로드
  model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
  # 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
  # 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

  sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
  return sentiment_model

In [36]:
sentiment_model = create_sentiment_bert()

sentiment_model.fit(train_x, train_y, epochs=3, shuffle=True, batch_size=100, validation_data=(test_x, test_y))

loading configuration file config.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading weights file model.safet

Epoch 1/3
221/221 [==============================] - 1946s 9s/step - loss: 0.6491 - accuracy: 0.6197 - val_loss: 0.5896 - val_accuracy: 0.6978
Epoch 2/3
221/221 [==============================] - 1867s 8s/step - loss: 0.5716 - accuracy: 0.7086 - val_loss: 0.5450 - val_accuracy: 0.7325
Epoch 3/3
221/221 [==============================] - 1878s 8s/step - loss: 0.5347 - accuracy: 0.7342 - val_loss: 0.5350 - val_accuracy: 0.7368


In [37]:
# PATH는 임의로 지정

path = "D:\\programming\\cloud-term-project"

In [38]:
sentiment_model.save_weights(path+"\\review_to_score.h5")

In [39]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []

    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

test 데이터 예측하기

In [40]:
test_set = predict_load_data(test)

100%|██████████| 5510/5510 [00:02<00:00, 2505.44it/s]


In [41]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')

    num_zeros = token.count(0)
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()

    print("만족도 수치화 ->", predict_value)
    return predict_value

In [42]:
evaluation_predict("히레카츠를 너무 좋아하는데 여기 진짜 찐찐찐 마쉿서여")

1/1 [==============================] - 4s 4s/step
만족도 수치화 -> [0.8757981]


array([0.8757981], dtype=float32)

In [43]:
evaluation_predict("맛있습니다. 웨이팅을 잘해야하지만, 버크셔k랑 맥주랑 조합이 미쳤습니다. 안시켰으면 후회할뻔")

1/1 [==============================] - 0s 102ms/step
만족도 수치화 -> [0.7579161]


array([0.7579161], dtype=float32)

In [44]:
evaluation_predict("바삭바삭하고 맛나요 버크셔랑 로스도 같이 먹었는데 다 맛있었어요")

1/1 [==============================] - 0s 171ms/step
만족도 수치화 -> [0.6937898]


array([0.6937898], dtype=float32)

In [46]:
evaluation_predict("나오자마자 먹었음에도 불구하고 아래쪽이 축축하게 젖어있었음")

1/1 [==============================] - 0s 132ms/step
만족도 수치화 -> [0.14313643]


array([0.14313643], dtype=float32)

In [47]:
evaluation_predict("생각보다 너무 실망함..돈까스 엄청 기대했는데 튀김도 고기도 기대 이하였음")

1/1 [==============================] - 0s 116ms/step
만족도 수치화 -> [0.07659423]


array([0.07659423], dtype=float32)

In [48]:
evaluation_predict("불향난다는 버크셔시켰는데 저는 개인적으로 불향이 별로였고 (향이 풍성해서 느끼함을 배로 증가 시키는 효과) 무엇보다 위쪽은 바삭한데 아랫쪽 튀김은 눅눅해서 먹기 거북")

1/1 [==============================] - 0s 101ms/step
만족도 수치화 -> [0.15108642]


array([0.15108642], dtype=float32)

In [49]:
evaluation_predict("제가 갔을때만 별로였나, 추천받아 간 곳이었는데 맛있는지도 잘 모르겠고.")

1/1 [==============================] - 0s 99ms/step
만족도 수치화 -> [0.19183655]


array([0.19183655], dtype=float32)